# BreCaNet

This Jupyter notebook goes through the steps to process the full data sets before running PANDA

##### Gene Expression Data

Came from the TCGA database: 
Settings for data: *Cases tab*
    Primary site -> breast,
    Disease type -> ductal and lobular neoplasms,
    Gender -> female,
    Race -> White
 
 *Files tab*
    Experimental Strategy -> RNA-seq,
    Workflow -> HTSeq-FPKM,
    Project -> TCGA-BRCA
    
Click on the Download button and select Cart

##### Clinical Data with IHC Receptor Status

Using the same settings as for gathering the expression data, except instead of downloading the cart, download the manifest data. The manifest file is used with the supplemental R code and necessary to retrieve the clinical IHC information.

Use the same settings in the TCGA repository on the Cases tab.

*Files tab:*
Data Category -> Clinical

Data Format -> BCR Biotab

Add to cart 

Download the nationawidechildrens.org_clinical_patient_brca.txt file

##### TF Binding Motif Data

GTex from Camila

##### Protien-Protein Interaction (PPI) Data

GTex from Camila

##### PANDA algorithm

https://netzoo.github.io

## Gets PAM50 labels of TCGA samples

###### How to get the labels for each sample:

1) Use TCGA label of the folder of the expression file and match with the file_id in the metadata

2) Use the case_id from the metadata that corresponds with the file_id to match the uuid in the clinical file

3) Use the uuid to get the corresponding barcode, also in the clinical file

4) Match the barcode from the clinical file to match with the barcode in the subtype label file to, finally, get the subtype of the sample

In [37]:
import numpy as np
import os
import shutil
import csv
import pandas as pd
import gzip
import json

In [25]:
# gene expression paths
dataPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/gdc_download_FPKM_832' #zipped file path
dataNewFolder = '/Users/ursulawidocki/Desktop/BreCaNet/Data/processedTCGA_832' # folder for unzipped
#PANDAexpPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAgeneExp_832.txt' # final exp file

In [26]:
# Unzips the expression data dn creates a folder (processedTCGA_832) of the unzipped files 
# Gets the codes of the patient folders and expression files

folderIDs = list() # codes of the patient IDs on the folder
indivList = list() # codes of the patients' expression file

for fileName in os.listdir(dataPath):
    if fileName == '.DS_Store' or fileName == 'MANIFEST.txt':
        continue
        
    else:
        folderIDs.append(fileName)
        
    newPath = dataPath + '/' + fileName
    
    # goes into the individual folder
    for indivName in os.listdir(newPath):
        if (indivName == '.DS_Store') or (indivName == 'MANIFEST.txt') or (indivName == 'annotations.txt'):
            continue
        filePath = newPath + '/' + indivName
        
        fileEnding = indivName[-3] + indivName[-2] + indivName[-1]
        if(fileEnding == '.gz'):
            with gzip.open(filePath, 'rb') as f_in:
                newFilePath = dataNewFolder + '/' + indivName[0:-3]
                indivList.append(indivName[0:-3])
                
                with open(newFilePath, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)

In [27]:
## Makes file of folder names

folderIDPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/foldIDList_FPKM_832.txt'
folderFile = open(folderIDPath, "w+")
for i in range(0, 832):
    newLineFolder = folderIDs[i] + "\n"
    
    folderFile.write(newLineFolder)
    
folderFile.close()

In [28]:
## Gets folder codes from expression data
folderIDPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/foldIDList_FPKM_832.txt'
folders = pd.read_csv(folderIDPath, delimiter = '\t', header = None)
folders_list = list(folders[0])

In [1]:
## Reads in metadata
# gets file_id and case_id

metaPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/metadata.cart.FPKM_832.json'

file_id = list()
case_id = list()
with open(metaPath, "r") as file:
    for line in file.readlines():
        
        if "file_id" in line:
            item = line.strip().split("\"")[3]
            #print(item)
                
        if("case_id" in line) and (line.strip().split("\"")[3] not in case_id):
            case = line.strip().split("\"")[3]
            case_id.append(case)
            file_id.append(item)

print(len(set(file_id)))
print(len(case_id))
print(len(set(case_id)))
#print(case_id[0:5])

714
714
714


In [30]:
## Gets uuid and barcodes of the patients from clinical data

# reads in clinical information to link patients with their barcode
clinicalPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/gdc_download_clinical_832/8162d394-8b64-4da2-9f5b-d164c54b9608/nationwidechildrens.org_clinical_patient_brca.txt'
clinical_df = pd.read_csv(clinicalPath, delimiter = '\t')

# removes rows that are not necessary to match data
clinical_df = clinical_df.drop(clinical_df.index[0])
clinical_df = clinical_df.drop(clinical_df.index[0])
clinical_df.head()
codes_df = clinical_df[['bcr_patient_uuid', 'bcr_patient_barcode']]
#codes_df.head()

uuid_list = list(codes_df['bcr_patient_uuid'])
barcode_list = list(codes_df['bcr_patient_barcode'])

print(len(uuid_list))
print(len(barcode_list))

1097
1097


In [31]:
## Reads in PAM50 labels

subtype_path = "/Users/ursulawidocki/Desktop/BreCaNet/Code/TCGA_subtypes.txt"
subtypes = pd.read_csv(subtype_path, delimiter = " ")
subtypes.head()

pt_barcodes_list = list(subtypes['brca_subtypes.patient'])
subtype_list = list(subtypes['brca_subtypes.BRCA_Subtype_PAM50'])

print(len(pt_barcodes_list))
print(len(subtype_list))

1087
1087


#### Matching data

In [35]:
## First, match the folder with the file_ids to get the corresponding cases from the metadata

match_cases = list()

for folder in folders_list:
    for file in file_id:
        if folder == file:
            ind = file_id.index(folder)
            match_cases.append(case_id[ind])
            break

print(len(match_cases)) # should be equivalent to number of samples

700


In [9]:
## Matches case_ids from metadata with uuid and patient barcodes from clinical data

match_barcodes = list()

for case in match_cases:
    for ptID in uuid_list:
        if case == ptID.lower():
            ind = uuid_list.index(ptID)
            match_barcodes.append(barcode_list[ind])
            break
            
print(len(match_barcodes))

832


In [10]:
## Matches subtypes to their barcodes from the TCGA data

match_subtypes = list()

for barcode in match_barcodes:
    ind = pt_barcodes_list.index(barcode)
    match_subtypes.append(subtype_list[ind])
    
print(len(match_subtypes))

832


In [11]:
print(len(set(match_subtypes))) # there is only this many subtypes among my samples
print((set(match_subtypes)))

5
{'LumA', 'Normal', 'LumB', 'Her2', 'Basal'}


## Now, sort the folder IDs into their subtypes

Notice that the order of folders_list is in the same order as match_subtypes

In [12]:
## uses index of subtype label to sort folder IDs into subtypes

# Luminal A samples
lumA_samples = list()
for i in range(0,len(match_subtypes)):
    if match_subtypes[i]=="LumA":
        lumA_samples.append(folders_list[i])

# Luminal B samples
lumB_samples = list()
for i in range(0,len(match_subtypes)):
    if match_subtypes[i]=="LumB":
        lumB_samples.append(folders_list[i])

# Basal samples
basal_samples = list()
for i in range(0,len(match_subtypes)):
    if match_subtypes[i]=="Basal":
        basal_samples.append(folders_list[i])

# Her2 samples
her2_samples = list()
for i in range(0,len(match_subtypes)):
    if match_subtypes[i]=="Her2":
        her2_samples.append(folders_list[i])

# Normal samples
norm_samples = list()
for i in range(0,len(match_subtypes)):
    if match_subtypes[i]=="Normal":
        norm_samples.append(folders_list[i])


### Now that the expression data is sorted, normalize and filter before running PANDA

In [13]:
# gets list of genes that are in the file

geneList = list() # list of all genes that are noted in the expression files

with open(dataNewFolder + '/' + indivList[0], "r") as file:
    for line in file.readlines():
        potentialGene = line.strip().split("\t")[0]
        
        if potentialGene[0:3] == "ENS":
            newGeneName = potentialGene.strip().split(".")[0]
            geneList.append(newGeneName)
print(len(geneList))

geneList_lumA = geneList
geneList_lumB = geneList
geneList_basal = geneList
geneList_her2 = geneList
geneList_norm = geneList

60483


In [14]:
## Reads through the unzipped expression files and creates the expression matrices depending on how the file
# name matches with the folder IDs of each subtype

lumA_exp = np.zeros((len(geneList), len(lumA_samples))) # exp matrix of LumA samples
lumB_exp = np.zeros((len(geneList), len(lumB_samples))) # exp matrix of LumB samples
basal_exp = np.zeros((len(geneList), len(basal_samples))) # exp matrix of basal samples
her2_exp = np.zeros((len(geneList), len(her2_samples))) # exp matrix of her2 samples
norm_exp = np.zeros((len(geneList), len(norm_samples))) # exp matrix of LumA samples

print(norm_exp.shape) # just to make sure dimensions are correct
 
lumA_j = 0 # index of the column (sample)index of the column (sample)
lumB_j = 0
basal_j = 0
her2_j = 0
norm_j = 0
for fileName in os.listdir(dataNewFolder):
    if fileName == '.DS_Store' or fileName == 'MANIFEST.txt':
        continue
    
    if fileName in indivList:
        
        temp_folder = folderIDs[indivList.index(fileName)]
        readFile = dataNewFolder + '/' + fileName
            
        # adds to luminal A expression matrix
        if temp_folder in lumA_samples:
            
            lumA_i = 0
            with open(readFile, "r") as file:
                for line in file.readlines():
                    lumA_exp[lumA_i, lumA_j] = float(line.strip().split("\t")[1])
                    lumA_i += 1
                    
                lumA_j += 1
                        
        # adds to luminal B expression matrix
        if temp_folder in lumB_samples:
            
            lumB_i = 0
            with open(readFile, "r") as file:
                for line in file.readlines():
                    lumB_exp[lumB_i, lumB_j] = float(line.strip().split("\t")[1])
                    lumB_i += 1
                    
                lumB_j += 1
        
        # adds to basal expression matrix
        if temp_folder in basal_samples:
            
            basal_i = 0
            with open(readFile, "r") as file:
                for line in file.readlines():
                    basal_exp[basal_i, basal_j] = float(line.strip().split("\t")[1])
                    basal_i += 1
                    
                basal_j += 1
        
        # adds to her2 expression matrix
        if temp_folder in her2_samples:
            
            her2_i = 0
            with open(readFile, "r") as file:
                for line in file.readlines():
                    her2_exp[her2_i, her2_j] = float(line.strip().split("\t")[1])
                    her2_i += 1
                    
                her2_j += 1
        
        # adds to norm expression matrix
        if temp_folder in norm_samples:
            
            norm_i = 0
            with open(readFile, "r") as file:
                for line in file.readlines():
                    norm_exp[norm_i, norm_j] = float(line.strip().split("\t")[1])
                    norm_i += 1
                    
                norm_j += 1
                    

(60483, 28)


In [15]:
## Removes zero rows from expression data

# Luminal A data
rowList_lumA = list()
for i in range(0, len(lumA_exp)):
    if np.sum(lumA_exp[i,:]) == 0.0:
        rowList_lumA.append(i)

lumA_exp2 = np.zeros(((len(lumA_exp)-len(rowList_lumA)), len(lumA_samples))) # matrix without zero rows
geneList_lumA2 = list()

j = 0 # index in old matrix
r = 0 # index in new matrix
while (j < len(lumA_exp)):
    if j in rowList_lumA:
        j+=1
    else:
        lumA_exp2[r,:] = lumA_exp[j,:]
        geneList_lumA2.append(geneList_lumA[j])
        j+=1
        r+=1
        
# Luminal B data
rowList_lumB = list()
for i in range(0, len(lumB_exp)):
    if np.sum(lumB_exp[i,:]) == 0.0:
        rowList_lumB.append(i)

lumB_exp2 = np.zeros(((len(lumB_exp)-len(rowList_lumB)), len(lumB_samples))) # matrix without zero rows
geneList_lumB2 = list()

j = 0 # index in old matrix
r = 0 # index in new matrix
while (j < len(lumB_exp)):
    if j in rowList_lumB:
        j+=1
    else:
        lumB_exp2[r,:] = lumB_exp[j,:]
        geneList_lumB2.append(geneList_lumB[j])
        j+=1
        r+=1
        
# Basal data
rowList_basal = list()
for i in range(0, len(basal_exp)):
    if np.sum(basal_exp[i,:]) == 0.0:
        rowList_basal.append(i)

basal_exp2 = np.zeros(((len(basal_exp)-len(rowList_basal)), len(basal_samples))) # matrix without zero rows
geneList_basal2 = list()

j = 0 # index in old matrix
r = 0 # index in new matrix
while (j < len(basal_exp)):
    if j in rowList_basal:
        j+=1
    else:
        basal_exp2[r,:] = basal_exp[j,:]
        geneList_basal2.append(geneList_basal[j])
        j+=1
        r+=1
        
# HER2 data
rowList_her2 = list()
for i in range(0, len(her2_exp)):
    if np.sum(her2_exp[i,:]) == 0.0:
        rowList_her2.append(i)

her2_exp2 = np.zeros(((len(her2_exp)-len(rowList_her2)), len(her2_samples))) # matrix without zero rows
geneList_her2_2 = list()

j = 0 # index in old matrix
r = 0 # index in new matrix
while (j < len(her2_exp)):
    if j in rowList_her2:
        j+=1
    else:
        her2_exp2[r,:] = her2_exp[j,:]
        geneList_her2_2.append(geneList_her2[j])
        j+=1
        r+=1

# Normal data
rowList_norm = list()
for i in range(0, len(norm_exp)):
    if np.sum(norm_exp[i,:]) == 0.0:
        rowList_norm.append(i)

norm_exp2 = np.zeros(((len(norm_exp)-len(rowList_norm)), len(norm_samples))) # matrix without zero rows
print(norm_exp2.shape)
geneList_norm2 = list()

print("OLD ", len(norm_exp))
print("ZEROS ", len(rowList_norm))
print("NEW ", len(norm_exp2))

j = 0 # index in old matrix
r = 0 # index in new matrix
while (j < len(norm_exp)):
    if j in rowList_norm:
        j+=1
    else:
        norm_exp2[r,:] = norm_exp[j,:]
        geneList_norm2.append(geneList_norm[j])
        j+=1
        r+=1
        
print(len(geneList_norm2))


(49832, 28)
OLD  60483
ZEROS  10651
NEW  49832
49832


In [16]:
## log(TPM) normalization
# TPMi = (FPKMi / sum(FPKMj)) * 10^6, where i is the gene and j is the sample
# logTPM = log(TPM + 1)

# Luminal A
for j in range(0, len(lumA_samples)):
    sumPat = np.sum(lumA_exp2, axis = 0)[j]
    
    for i in range(0, len(lumA_exp2)):
        lumA_exp2[i,j] = np.log((lumA_exp2[i,j] / sumPat) * (10**6) + 1)
        
# Luminal B
for j in range(0, len(lumB_samples)):
    sumPat = np.sum(lumB_exp2, axis = 0)[j]
    
    for i in range(0, len(lumB_exp2)):
        lumB_exp2[i,j] = np.log((lumB_exp2[i,j] / sumPat) * (10**6) + 1)
        
# Basal
for j in range(0, len(basal_samples)):
    sumPat = np.sum(basal_exp2, axis = 0)[j]
    
    for i in range(0, len(basal_exp2)):
        basal_exp2[i,j] = np.log((basal_exp2[i,j] / sumPat) * (10**6) + 1)
        
# HER2
for j in range(0, len(her2_samples)):
    sumPat = np.sum(her2_exp2, axis = 0)[j]
    
    for i in range(0, len(her2_exp2)):
        her2_exp2[i,j] = np.log((her2_exp2[i,j] / sumPat) * (10**6) + 1)
        
# Normal
for j in range(0, len(norm_samples)):
    sumPat = np.sum(norm_exp2, axis = 0)[j]
    
    for i in range(0, len(norm_exp2)):
        norm_exp2[i,j] = np.log((norm_exp2[i,j] / sumPat) * (10**6) + 1)


In [17]:
## Extracts TFBM information
# keeps track of which TF and genes have relations as per TFBM file

motifPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/motif.txt' # original motif file

motifTF_lumA = list() # list of TFs interacting to genes that are expressed
motifGenes_lumA = list() # list of genes in expression file with TFs
motifTF_lumB = list()
motifGenes_lumB = list()
motifTF_basal = list()
motifGenes_basal = list()
motifTF_her2 = list()
motifGenes_her2 = list()
motifTF_norm = list()
motifGenes_norm = list()

with open(motifPath, "r") as file:
    for line in file.readlines():
        if line.strip().split("\t")[2] == "1":
            
            # gets motif data for Luminal A
            if line.strip().split("\t")[1] in geneList_lumA2:
                motifTF_lumA.append(line.strip().split("\t")[0])
                motifGenes_lumA.append(line.strip().split("\t")[1])
                
            # gets motif data for Luminal B
            if line.strip().split("\t")[1] in geneList_lumB2:
                motifTF_lumB.append(line.strip().split("\t")[0])
                motifGenes_lumB.append(line.strip().split("\t")[1])
                
            # gets motif data for Basal
            if line.strip().split("\t")[1] in geneList_basal:
                motifTF_basal.append(line.strip().split("\t")[0])
                motifGenes_basal.append(line.strip().split("\t")[1])
                
            # gets motif data for HER2
            if line.strip().split("\t")[1] in geneList_her2:
                motifTF_her2.append(line.strip().split("\t")[0])
                motifGenes_her2.append(line.strip().split("\t")[1])
                
            # gets motif data for Normal
            if line.strip().split("\t")[1] in geneList_norm:
                motifTF_norm.append(line.strip().split("\t")[0])
                motifGenes_norm.append(line.strip().split("\t")[1])

In [18]:
print(len(motifTF_lumA))
print(len(motifGenes_lumA))

print(len(motifTF_lumB))
print(len(motifGenes_lumB))

print(len(motifTF_basal))
print(len(motifGenes_basal))

print(len(motifTF_her2))
print(len(motifGenes_her2))

print(len(motifTF_norm))
print(len(motifGenes_norm))

1792566
1792566
1790056
1790056
1806322
1806322
1806322
1806322
1806322
1806322


In [19]:
## Checks if there are genes expressed and not in the motif file
# if not, then doesn't add to updated list of genes in the exp matrix
# and makes note of it to remove later

# Luminal A
removeGene_lumA = list() # list of indeces to remove from expressed genes
for i in range(0, len(geneList_lumA2)):
    if geneList_lumA2[i] not in motifGenes_lumA:
        removeGene_lumA.append(i)
        
print(len(removeGene_lumA))

## Luminal B
removeGene_lumB = list() # list of indeces to remove from expressed genes
for i in range(0, len(geneList_lumB2)):
    if geneList_lumB2[i] not in motifGenes_lumB:
        removeGene_lumB.append(i)
        
print(len(removeGene_lumB))

# Basal
removeGene_basal = list() # list of indeces to remove from expressed genes
for i in range(0, len(geneList_basal2)):
    if geneList_basal2[i] not in motifGenes_basal:
        removeGene_basal.append(i)
        
print(len(removeGene_basal))

# HER2
removeGene_her2 = list() # list of indeces to remove from expressed genes
for i in range(0, len(geneList_her2_2)):
    if geneList_her2_2[i] not in motifGenes_her2:
        removeGene_her2.append(i)
        
print(len(removeGene_her2))

# Normal
removeGene_norm = list() # list of indeces to remove from expressed genes
for i in range(0, len(geneList_norm2)):
    if geneList_norm2[i] not in motifGenes_norm:
        removeGene_norm.append(i)
        
print(len(removeGene_norm))


28936
26929
27078
24096
21579


In [20]:
## Removes the rows and genes that are not in the motif file

# Luminal A
lumA_exp3 = np.zeros((len(lumA_exp2)-len(removeGene_lumA), len(lumA_samples)))
geneList_lumA3 = list() # list of genes that are in motif file and the motifs are expressed genes

r = 0 # index of row in allTesting3
for i in range(0, len(lumA_exp2)):
    if i not in removeGene_lumA:
        lumA_exp3[r,:] = lumA_exp2[i,:]
        geneList_lumA3.append(geneList_lumA2[i])
        r+=1

# Luminal B
lumB_exp3 = np.zeros((len(lumB_exp2)-len(removeGene_lumB), len(lumB_samples)))
geneList_lumB3 = list() # list of genes that are in motif file and the motifs are expressed genes

r = 0 # index of row in allTesting3
for i in range(0, len(lumB_exp2)):
    if i not in removeGene_lumB:
        lumB_exp3[r,:] = lumB_exp2[i,:]
        geneList_lumB3.append(geneList_lumB2[i])
        r+=1

# Basal
basal_exp3 = np.zeros((len(basal_exp2)-len(removeGene_basal), len(basal_samples)))
geneList_basal3 = list() # list of genes that are in motif file and the motifs are expressed genes

r = 0 # index of row in allTesting3
for i in range(0, len(basal_exp2)):
    if i not in removeGene_basal:
        basal_exp3[r,:] = basal_exp2[i,:]
        geneList_basal3.append(geneList_basal2[i])
        r+=1
        
# HER2
her2_exp3 = np.zeros((len(her2_exp2)-len(removeGene_her2), len(her2_samples)))
geneList_her2_3 = list() # list of genes that are in motif file and the motifs are expressed genes

r = 0 # index of row in allTesting3
for i in range(0, len(her2_exp2)):
    if i not in removeGene_her2:
        her2_exp3[r,:] = her2_exp2[i,:]
        geneList_her2_3.append(geneList_her2_2[i])
        r+=1
        
# Normal
norm_exp3 = np.zeros((len(norm_exp2)-len(removeGene_norm), len(norm_samples)))
geneList_norm3 = list() # list of genes that are in motif file and the motifs are expressed genes

r = 0 # index of row in allTesting3
for i in range(0, len(norm_exp2)):
    if i not in removeGene_norm:
        norm_exp3[r,:] = norm_exp2[i,:]
        geneList_norm3.append(geneList_norm2[i])
        r+=1
        
print(norm_exp3.shape)
print(len(geneList_norm3))

(28253, 28)
28253


In [23]:
## Removes genes the are not expressed but in motif file

# gets all indeces of an element(gene) in a list(motifGenes)
def getIndeces(listOfElements, element):
    indexPosList = []
    indexPos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            indexPos = listOfElements.index(element, indexPos)
            # Add the index position in list
            indexPosList.append(indexPos)
            indexPos += 1
        except ValueError as e:
            break
 
    return indexPosList

# Luminal A
if set(motifGenes_lumA) != set(geneList_lumA3):
    print(len(set(motifGenes_lumA)))
    print(len(set(geneList_lumA3)))
    
    diff = list(set(motifGenes_lumA) - set(geneList_lumA3))
    print(len(diff))
    for gene in diff:
        geneInds = getIndeces(motifGenes_lumA, gene)

        for i in range(0, len(geneInds)):
            motifGenes_lumA.remove(motifGenes_lumA[geneInds[i] - i])
            motifTF_lumA.remove(motifTF_lumA[geneInds[i] - i])

# Luminal B
if set(motifGenes_lumB) != set(geneList_lumB3):
    print(len(set(motifGenes_lumB)))
    print(len(set(geneList_lumB3)))
    
    diff = list(set(motifGenes_lumB) - set(geneList_lumB3))
    print(len(diff))
    for gene in diff:
        geneInds = getIndeces(motifGenes_lumB, gene)

        for i in range(0, len(geneInds)):
            motifGenes_lumB.remove(motifGenes_lumB[geneInds[i] - i])
            motifTF_lumB.remove(motifTF_lumB[geneInds[i] - i])

# Basal
if set(motifGenes_basal) != set(geneList_basal3):
    print(len(set(motifGenes_basal)))
    print(len(set(geneList_basal3)))
    
    diff = list(set(motifGenes_basal) - set(geneList_basal3))
    print(len(diff))
    for gene in diff:
        geneInds = getIndeces(motifGenes_basal, gene)

        for i in range(0, len(geneInds)):
            motifGenes_basal.remove(motifGenes_basal[geneInds[i] - i])
            motifTF_basal.remove(motifTF_basal[geneInds[i] - i])
            
print(set(motifGenes_basal)== set(geneList_basal3))

# HER2
if set(motifGenes_her2) != set(geneList_her2_3):
    print(len(set(motifGenes_her2)))
    print(len(set(geneList_her2_3)))
    
    diff = list(set(motifGenes_her2) - set(geneList_her2_3))
    print(len(diff))
    for gene in diff:
        geneInds = getIndeces(motifGenes_her2, gene)

        for i in range(0, len(geneInds)):
            motifGenes_her2.remove(motifGenes_her2[geneInds[i] - i])
            motifTF_her2.remove(motifTF_her2[geneInds[i] - i])
            
print(set(motifGenes_her2)== set(geneList_her2_3))

# Normal
if set(motifGenes_norm) != set(geneList_norm3):
    print(len(set(motifGenes_norm)))
    print(len(set(geneList_norm3)))
    
    diff = list(set(motifGenes_norm) - set(geneList_norm3))
    print(len(diff))
    for gene in diff:
        geneInds = getIndeces(motifGenes_norm, gene)

        for i in range(0, len(geneInds)):
            motifGenes_norm.remove(motifGenes_norm[geneInds[i] - i])
            motifTF_norm.remove(motifTF_norm[geneInds[i] - i])
            
print(set(motifGenes_norm)== set(geneList_norm3))


True
29374
28552
822
True
29374
28253
1121
True


In [33]:
print(len(motifTF_basal))
print(len(motifGenes_basal))

print(len(motifTF_her2))
print(len(motifGenes_her2))

print(len(motifTF_norm))
print(len(motifGenes_norm))

1790318
1790318
1782492
1782492
1772784
1772784


In [32]:
## Reads in the ppi data and extracts the proteins 
# About 4 hrs to run

ppiPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/ppi2015_freeze.txt'

protein1_lumA = list()
protein2_lumA = list()
protein1_lumB = list()
protein2_lumB = list()
protein1_basal = list()
protein2_basal = list()
protein1_her2 = list()
protein2_her2 = list()
protein1_norm = list()
protein2_norm = list()

with open(ppiPath, "r") as file:
    next(file) #skips the first line since that is just a header
    for line in file.readlines():
        if line.strip().split("\t")[0] in motifTF_lumA:
            protein1_lumA.append(line.strip().split("\t")[0])
            protein2_lumA.append(line.strip().split("\t")[1])
        
        if line.strip().split("\t")[0] in motifTF_lumB:
            protein1_lumB.append(line.strip().split("\t")[0])
            protein2_lumB.append(line.strip().split("\t")[1])
            
        if line.strip().split("\t")[0] in motifTF_basal:
            protein1_basal.append(line.strip().split("\t")[0])
            protein2_basal.append(line.strip().split("\t")[1])
            
        if line.strip().split("\t")[0] in motifTF_her2:
            protein1_her2.append(line.strip().split("\t")[0])
            protein2_her2.append(line.strip().split("\t")[1])
            
        if line.strip().split("\t")[0] in motifTF_norm:
            protein1_norm.append(line.strip().split("\t")[0])
            protein2_norm.append(line.strip().split("\t")[1])

In [34]:
# Check to see if the two protein lists have the same proteins or not

if(set(protein1_lumA) == set(protein2_lumA)):
    print("Same!!")
    
if(set(protein1_lumB) == set(protein2_lumB)):
    print("Same!!")
    
if(set(protein1_basal) == set(protein2_basal)):
    print("Same!!")
    
if(set(protein1_her2) == set(protein2_her2)):
    print("Same!!")
    
if(set(protein1_norm) == set(protein2_norm)):
    print("Same!!")
    
# all proteins sets contain the same proteins

Same!!
Same!!
Same!!
Same!!
Same!!


In [35]:
## Makes the expression filess

# Luminal A
PANDAexpPath_lumA = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAgeneExp_lumA.txt'
geneExpFile = open(PANDAexpPath_lumA,"w+")

for i in range(0, len(geneList_lumA3)):
    newLine = geneList_lumA3[i]
    
    for j in range(0, len(lumA_samples)):
        newLine = newLine + "\t" + str(lumA_exp3[i,j])
        
    newLine = newLine + "\n"
    geneExpFile.write(newLine)
    
geneExpFile.close()

# Luminal B
PANDAexpPath_lumB = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAgeneExp_lumB.txt'
geneExpFile = open(PANDAexpPath_lumB,"w+")

for i in range(0, len(geneList_lumB3)):
    newLine = geneList_lumB3[i]
    
    for j in range(0, len(lumB_samples)):
        newLine = newLine + "\t" + str(lumB_exp3[i,j])
        
    newLine = newLine + "\n"
    geneExpFile.write(newLine)
    
geneExpFile.close()

# Basal
PANDAexpPath_basal = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAgeneExp_basal.txt'
geneExpFile = open(PANDAexpPath_basal,"w+")

for i in range(0, len(geneList_basal3)):
    newLine = geneList_basal3[i]
    
    for j in range(0, len(basal_samples)):
        newLine = newLine + "\t" + str(basal_exp3[i,j])
        
    newLine = newLine + "\n"
    geneExpFile.write(newLine)
    
geneExpFile.close()

# HER2
PANDAexpPath_her2 = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAgeneExp_her2.txt'
geneExpFile = open(PANDAexpPath_her2,"w+")

for i in range(0, len(geneList_her2_3)):
    newLine = geneList_her2_3[i]
    
    for j in range(0, len(her2_samples)):
        newLine = newLine + "\t" + str(her2_exp3[i,j])
        
    newLine = newLine + "\n"
    geneExpFile.write(newLine)
    
geneExpFile.close()

# Normal
PANDAexpPath_norm = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAgeneExp_norm.txt'
geneExpFile = open(PANDAexpPath_norm,"w+")

for i in range(0, len(geneList_norm3)):
    newLine = geneList_norm3[i]
    
    for j in range(0, len(norm_samples)):
        newLine = newLine + "\t" + str(norm_exp3[i,j])
        
    newLine = newLine + "\n"
    geneExpFile.write(newLine)
    
geneExpFile.close()

In [36]:
## Makes the motif files

# Luminal A
PANDAmotifPath_lumA = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAmotifs_lumA.txt'
motifFile = open(PANDAmotifPath_lumA, "w+")

for i in range(0, len(motifTF_lumA)):
    newLine = motifTF_lumA[i] + "\t" + motifGenes_lumA[i] + "\t" + "1.000000" + "\n"
    motifFile.write(newLine)

motifFile.close()

# Luminal B
PANDAmotifPath_lumB = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAmotifs_lumB.txt'
motifFile = open(PANDAmotifPath_lumB, "w+")

for i in range(0, len(motifTF_lumB)):
    newLine = motifTF_lumB[i] + "\t" + motifGenes_lumB[i] + "\t" + "1.000000" + "\n"
    motifFile.write(newLine)

motifFile.close()

# Basal
PANDAmotifPath_basal = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAmotifs_basal.txt'
motifFile = open(PANDAmotifPath_basal, "w+")

for i in range(0, len(motifTF_basal)):
    newLine = motifTF_basal[i] + "\t" + motifGenes_basal[i] + "\t" + "1.000000" + "\n"
    motifFile.write(newLine)

motifFile.close()

# HER2
PANDAmotifPath_her2 = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAmotifs_her2.txt'
motifFile = open(PANDAmotifPath_her2, "w+")

for i in range(0, len(motifTF_her2)):
    newLine = motifTF_her2[i] + "\t" + motifGenes_her2[i] + "\t" + "1.000000" + "\n"
    motifFile.write(newLine)

motifFile.close()

# Normal
PANDAmotifPath_norm = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAmotifs_norm.txt'
motifFile = open(PANDAmotifPath_norm, "w+")

for i in range(0, len(motifTF_norm)):
    newLine = motifTF_norm[i] + "\t" + motifGenes_norm[i] + "\t" + "1.000000" + "\n"
    motifFile.write(newLine)

motifFile.close()

In [37]:
## Makes the ppi files

# Luminal A
PANDAppiPath_lumA = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAppi_lumA.txt'
ppiFile = open(PANDAppiPath_lumA, "w+")

for i in range(0, len(protein1_lumA)):
    newLine = protein1_lumA[i] + "\t" + protein2_lumA[i] + "\t" + "1" + "\n"
    ppiFile.write(newLine)

ppiFile.close()

# Luminal B
PANDAppiPath_lumB = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAppi_lumB.txt'
ppiFile = open(PANDAppiPath_lumB, "w+")

for i in range(0, len(protein1_lumB)):
    newLine = protein1_lumB[i] + "\t" + protein2_lumB[i] + "\t" + "1" + "\n"
    ppiFile.write(newLine)

ppiFile.close()

# Basal
PANDAppiPath_basal = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAppi_basal.txt'
ppiFile = open(PANDAppiPath_basal, "w+")

for i in range(0, len(protein1_basal)):
    newLine = protein1_basal[i] + "\t" + protein2_basal[i] + "\t" + "1" + "\n"
    ppiFile.write(newLine)

ppiFile.close()

# HER2
PANDAppiPath_her2 = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAppi_her2.txt'
ppiFile = open(PANDAppiPath_her2, "w+")

for i in range(0, len(protein1_her2)):
    newLine = protein1_her2[i] + "\t" + protein2_her2[i] + "\t" + "1" + "\n"
    ppiFile.write(newLine)

ppiFile.close()

# Normal
PANDAppiPath_norm = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAppi_norm.txt'
ppiFile = open(PANDAppiPath_norm, "w+")

for i in range(0, len(protein1_norm)):
    newLine = protein1_norm[i] + "\t" + protein2_norm[i] + "\t" + "1" + "\n"
    ppiFile.write(newLine)

ppiFile.close()